In [3]:
import pandas as pd
df = pd.read_csv('movie_tags.csv')
L = df.head(20)['title'].tolist()

In [7]:
import json
from tqdm.auto import trange
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain.prompts.chat import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
import openai
from langchain.chat_models import ChatOpenAI

OPENAI_API_BASE= "https://tcl-ai.openai.azure.com"
OPENAI_API_TYPE = "azure"
OPENAI_API_VERSION = '2023-07-01-preview'
OPENAI_API_KEY= "6e25ec6fa59d44f8af091db59e6db6d7"
OPENAI_DEPLOYMENT_NAME = 'gpt-4-0314'

openai.api_key = OPENAI_API_KEY
openai.api_base = OPENAI_API_BASE
openai.api_type = OPENAI_API_TYPE
openai.api_version = OPENAI_API_VERSION
chat = ChatOpenAI(engine=OPENAI_DEPLOYMENT_NAME, openai_api_key=OPENAI_API_KEY)

theme_output= '''
    {“目标电影”:“”,
    “标签”:“”, 
    “得分”:“”}
'''

theme_template= '''
    候选列表：{theme_list}
    输出：剧情节奏
    
    请逐步思考
    1.参考候选列表，结合你的知识(豆瓣,百度百科,维基百科等)，生成输出目标电影的剧情节奏，如果你不知道，请返回‘其他’
    2.重复上述操作三次。
    3.请针对上述所有标签，逐个判断是否有明确依据，输出符合程度‘高’，‘中’，‘低’。
    4.仅保留符合程度为‘高’的标签，根据依据来源投票打分并排序（10分制），保留大于9分的标签；如果没有大于9分的标签，则仅保留top1的标签。
    5.输出json格式，格式参考{output}。
    6.如果步骤5生成的结果中，参考列表未出现且没有相似标签时，请添加到参考列表，并返回最新的参考列表
    answer the users question as best as possible.
    {format_instructions}
'''

theme_list = '''
节奏缓慢，渐进节奏，节奏紧凑，剧情反转，紧张刺激，高潮迭起，悬念迭起，悬疑重重，回溯叙述，思考深度，情感共鸣，多线索，节奏明快, 轻松幽默
'''

response_schemas = [
    ResponseSchema(name="1.generate one tag", description="should be a string"),
    ResponseSchema(name="2.repeat 1 for 3 times", description=""),
    ResponseSchema(name="3.judgement", description=""),
    ResponseSchema(name="4.filter tags score and rank", description=""),
    ResponseSchema(name="5.output", description="should be a json format"),
]


if __name__ =='__main__':
   
    results=dict()
    for movie in L:
        target_movie = movie

        output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
        format_instructions = output_parser.get_format_instructions()

        #输入格式
        system_message_prompt = SystemMessagePromptTemplate.from_template(theme_template)
        human_template="目标电影：{target_movie}"
        human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)
        prompt = ChatPromptTemplate(
            messages=[system_message_prompt, human_message_prompt],
            input_variables=["theme_list","target_movie","output"],
            partial_variables={"format_instructions": format_instructions}
        )

        _input = prompt.format_prompt(theme_list=theme_list,target_movie=target_movie,output=theme_output)
        output = chat(_input.to_messages())
        output = output_parser.parse(output.content)
        try:
            data = json.loads(output['5.output'])
        except:
            data  = output['5.output']
            
        print(data)

WARNING! engine is not default parameter.
                    engine was transferred to model_kwargs.
                    Please confirm that engine is what you intended.


{'目标电影': '封神三部曲', '标签': '剧情反转, 紧张刺激, 悬念迭起', '得分': '9.5, 9, 9'}
{'目标电影': '长安三万里', '标签': '节奏缓慢', '得分': '10'}
{'目标电影': '碟中谍7：致命清算（上）', '标签': '节奏紧凑, 紧张刺激, 高潮迭起', '得分': '10, 10, 10'}
{'目标电影': '变形金刚：超能勇士崛起', '标签': '高潮迭起, 紧张刺激, 节奏紧凑', '得分': '10, 10, 10'}
{'目标电影': '消失的她', '标签': '节奏紧凑, 悬念迭起, 悬疑重重', '得分': '10, 10, 10'}
{“目标电影”:“速度与激情10”, “标签”:“节奏紧凑, 紧张刺激, 高潮迭起”, “得分”:“10, 10, 10”}
{'目标电影': '哆啦A梦：大雄的新恐龙', '标签': '节奏明快, 轻松幽默', '得分': '10, 10'}
{'目标电影': '我的爸爸是森林之王2', '标签': '节奏明快, 轻松幽默', '得分': '9.5, 9.2'}
{'目标电影': '芭比公主历险记', '标签': '节奏明快, 轻松幽默', '得分': '10, 10'}
{'目标电影': '暗杀风暴', '标签': '节奏紧凑, 紧张刺激, 悬疑重重', '得分': '10, 9.5, 9'}
{'目标电影': '汪汪队立大功大电影', '标签': '节奏明快, 轻松幽默', '得分': '10, 10'}
{'目标电影': '机械师', '标签': '节奏紧凑,紧张刺激,高潮迭起', '得分': '10,9.5,9'}
{'目标电影': '铃芽之旅', '标签': '节奏缓慢, 情感共鸣', '得分': '9.5, 9.0'}
{'目标电影': '熊出没·狂野大陆', '标签': '节奏明快, 轻松幽默', '得分': '10, 10'}
{'目标电影': '山炮进城', '标签': '节奏明快, 轻松幽默', '得分': '9.5, 9.5'}
{'目标电影': '萌鸡小队：萌闯新世界', '标签': '节奏明快, 轻松幽默', '得分': '10, 10'}
{'目标电影': '银行家', '标签': '紧张刺激,悬疑重重', '得分': '10

In [16]:
# 输入字符串


theme_list: 真实改编,小说改编,漫画动画改编,翻拍,系列续集,原创剧本,故事改编,游戏改编,历史改编,人物传记,其他
theme_output: {"目标电影":''", "改编来源":""}。
theme_template: 
候选列表：{theme_list}
输出：
改编来源
请逐步思考
1.参考以上参考列表，输出目标电影的改变来源，如果你不知道，请返回‘其他’
2.重复上述操作三次。
3.请针对上述所有标签，逐个判断是否有明确依据，输出程度‘高’，‘中’，‘低’。
4.保留程度为‘高’的标签，根据依据来源投票打分并排序（10分制），只保留Top 1的一个标签。
5.最后判断保留的标签是否来源准确，如果准确，则输出该标签，否则输出‘其他’。输出json格式，格式参考{output}
answer the users question as best as possible.
{format_instructions}


In [39]:
import json
from tqdm.auto import trange
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain.prompts.chat import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
import openai
from langchain.chat_models import ChatOpenAI
import pandas as pd
df = pd.read_csv('movie_tags.csv')
L = df.head(20)['title'].tolist() # 取前20个媒资

OPENAI_API_BASE= "https://tcl-ai.openai.azure.com"
OPENAI_API_TYPE = "azure"
OPENAI_API_VERSION = '2023-07-01-preview'
OPENAI_API_KEY= "6e25ec6fa59d44f8af091db59e6db6d7"
OPENAI_DEPLOYMENT_NAME = 'gpt-4-0314'

openai.api_key = OPENAI_API_KEY
openai.api_base = OPENAI_API_BASE
openai.api_type = OPENAI_API_TYPE
openai.api_version = OPENAI_API_VERSION
chat = ChatOpenAI(engine=OPENAI_DEPLOYMENT_NAME, openai_api_key=OPENAI_API_KEY)

with open("prompts.txt", "r", encoding="utf-8") as file:
    content = file.read()
prompts = content.split("———————————————————————————————————————")
prompts = [prompt.strip() for prompt in prompts if prompt.strip()]

# txt文件中提取input要求
input_string = prompts[0]
theme_list = input_string.split("参考列表：")[1].split("]")[0].replace("[", "").strip()
theme_output = input_string.split("输出json格式，格式参考")[1].split(".")[0].strip()
theme_template = '\n候选列表：{theme_list}\n输出：\n' + input_string.split("输出：")[1].replace(theme_output, '{output}').strip() + "\nanswer the users question as best as possible.\n{format_instructions}"


theme_output= theme_output
theme_template= theme_template
theme_list = theme_list


response_schemas = [
    ResponseSchema(name="1.generate one tag", description="should be a string"),
    ResponseSchema(name="2.repeat 1 for 3 times", description=""),
    ResponseSchema(name="3.judgement", description=""),
    ResponseSchema(name="4.filter tags score and rank", description=""),
    ResponseSchema(name="5.output", description="should be a json format"),
]


if __name__ =='__main__':
    
    df = pd.DataFrame()
    
    for movie in L[:2]:
        target_movie = movie

        output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
        format_instructions = output_parser.get_format_instructions()

        system_message_prompt = SystemMessagePromptTemplate.from_template(theme_template)
        human_template="目标电影：{target_movie}"
        human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)
        prompt = ChatPromptTemplate(
            messages=[system_message_prompt, human_message_prompt],
            input_variables=["theme_list", "target_movie", "output"],
            partial_variables={"format_instructions": format_instructions}
        )

        _input = prompt.format_prompt(theme_list=theme_list, target_movie=target_movie, output=theme_output)
        output = chat(_input.to_messages())
        output = output_parser.parse(output.content)
        
        try:
            data = json.loads(output['5.output'])
        except:
            data  = output['5.output']
            
        df = pd.concat([df, pd.DataFrame([data])])

    df.to_csv("output.csv")

WARNING! engine is not default parameter.
                    engine was transferred to model_kwargs.
                    Please confirm that engine is what you intended.


In [40]:
df

,目标电影,标签,得分
0,封神三部曲,节奏紧凑,10
0,长安三万里,"节奏紧凑, 剧情反转, 悬疑重重","10, 9.5, 9"


In [41]:
! python3 promptForExtract.py

WARNING! engine is not default parameter.
                    engine was transferred to model_kwargs.
                    Please confirm that engine is what you intended.


,目标电影,改编来源
0,长安三万里,历史改编
